In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
import csv
import uuid
import os
import streamlit as st
from PIL import Image

# Your previous functions and COLOR dictionary here...

# Load CSV file with image details and color codes
csv_filename = 'output_info_with_class.csv'  # Replace with your CSV file containing image information
df = pd.read_csv(csv_filename)

# Streamlit app layout
st.title("Image Retrieval")

# File uploader for image prediction
file = st.file_uploader("Upload an image:", type=["jpg", "jpeg", "png"])

if file is not None:
    image = Image.open(file)
    st.image(image, caption='Uploaded Image', use_column_width=True)

    # Extract dominant color from the uploaded image
    img = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    pixels = img.reshape((-1, 3))
    kmeans = KMeans(n_clusters=1)
    kmeans.fit(pixels)
    dominant_color = kmeans.cluster_centers_.astype(int)[0]
    closest_color = get_closest_color(dominant_color)

    # Predict the closest matching class based on the closest color
    selected_class = st.selectbox("Predicted Class:", df['Class'].unique())

    # Filter data based on predicted class and closest color
    filtered_data = df[(df['Class'] == selected_class) & (df['Dominant Color'] == closest_color)]

    if not filtered_data.empty:
        found_image = False
        matching_images = filtered_data['Image ID']

        # Display images based on detected class and closest color
        for image_id in matching_images:
            row = df[df['Image ID'] == image_id]
            image_path = os.path.join('100_Images', f"{image_id}.jpg")  # Assuming images are in a folder named '100_Images'
            if os.path.exists(image_path):
                img = Image.open(image_path)
                st.image(img, use_column_width=True)
                found_image = True
                break

        # If no matching image with the closest color, display any random image of the class
        if not found_image:
            random_image_id = random.choice(matching_images)
            image_path = os.path.join('100_Images', f"{random_image_id}.jpg")
            if os.path.exists(image_path):
                img = Image.open(image_path)
                st.image(img, use_column_width=True)

    else:
        st.write("No matching image found.")
